In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess, CovarianceStructure, ObjectiveLib
import matplotlib.pyplot as plt
import pickle 

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

Names in str: 'Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)'

In [2]:
Nt =110

max_manual_num = 5 
min_interval_num = 10

In [3]:
static_ind = [0,1,2,3,4,5,6,7,8,9,10]
dynamic_ind = [11,12,13,14,15]
all_ind = static_ind+dynamic_ind

num_total = len(all_ind)

In [4]:
all_names_strategy3 = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T',  'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', # all static
            'Ads.gas_outlet.z("CO2").static', 'Des.gas_outlet.z("CO2").static', # static z 
            'Ads.gas_outlet.z("CO2").dynamic', 'Des.gas_outlet.z("CO2").dynamic', # dynamic z 
            'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [5]:

static_cost = [1000, #ads.gas_inlet.F (0)
    1000, #ads.gas_outlet.F (1)
     500, #ads.gas_outlet.T (2)
    1000, #des.gas_inlet.F (4)
    1000, #des.gas_outlet.F (5)
     500, #des.gas_outlet.T (6)
     1000, #ads.T19 (8)
     1000, #ads.T23 (9)
     1000, #ads.T28 (10)
    7000,
    7000]


static_cost.extend([100, 100, 500, 500, 500])

dynamic_cost = [0]*len(static_ind)
dynamic_cost.extend([100]*len(dynamic_ind))

max_manual = [max_manual_num]*num_total
min_time_interval = [min_interval_num]*num_total

In [6]:
print("if selected all static:", sum(static_cost))
print("if selected all static without two z:", sum(static_cost)-7000*2)

if selected all static: 23700
if selected all static without two z: 9700


In [7]:
measure_info = pd.DataFrame({
    "name": all_names_strategy3,
    "Q_index": all_ind,
        "static_cost": static_cost,
    "dynamic_cost": dynamic_cost,
    "min_time_interval": min_time_interval, 
    "max_manual_number": max_manual
})

print(measure_info)

                               name  Q_index  static_cost  dynamic_cost  \
0                   Ads.gas_inlet.F        0         1000             0   
1                  Ads.gas_outlet.F        1         1000             0   
2                  Ads.gas_outlet.T        2          500             0   
3                   Des.gas_inlet.F        3         1000             0   
4                  Des.gas_outlet.F        4         1000             0   
5                  Des.gas_outlet.T        5          500             0   
6              Ads.T_g.Value(19,10)        6         1000             0   
7              Ads.T_g.Value(23,10)        7         1000             0   
8              Ads.T_g.Value(28,10)        8         1000             0   
9    Ads.gas_outlet.z("CO2").static        9         7000             0   
10   Des.gas_outlet.z("CO2").static       10         7000             0   
11  Ads.gas_outlet.z("CO2").dynamic       11          100           100   
12  Des.gas_outlet.z("CO2

In [8]:
print(measure_info[measure_info['name']=='Ads.gas_outlet.z("CO2").static'].index.values)
print(len(measure_info))

[9]
16


In [9]:
dataObject = DataProcess()
dataObject.read_jacobian('./RotaryBed/Q'+str(Nt)+'_scale.csv')
Q = dataObject.get_Q_list([0,1,2,4,5,6,8,9,10,3,7],
                        [3,7,11,12,13], Nt)


In [10]:
calculator = MeasurementOptimizer(Q, measure_info, error_opt=CovarianceStructure.identity, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(314721, 5, 5)


### Debug functions: check each unit FIM

In [ ]:
print(calculator.head_pos_dynamic_flatten)
print(calculator.static_idx)
print(calculator.dynamic_idx)
print(calculator.Q_dynamic_flatten[451][0])

In [ ]:
# check Q is rightly arranged 
for i in range(16):
    print(Q[i][0])

In [ ]:
def check(fim_set):
    for i in range(num_total):
        for j in range(num_total):
            fim = fim_set[i*num_total+j]
            if np.any(fim):
                logdet = np.log(np.linalg.det(fim))
                if logdet < 0:
                    print("unit FIM with small Det: ", i, j, ', logdet:', logdet)
                if max(np.linalg.eigvals(fim))/min(np.linalg.eigvals(fim)) > 1000:
                    print("unit FIM with big condition number:", i, j, ", eigvals:", np.linalg.eigvals(fim))
                    
def print_fim(t):
    fim = calculator.fim_collection[t*num_total+t]
    print("Measurement name: ", all_names_strategy3[t])
    #print(fim)
    print("Trace:", np.trace(fim))

In [ ]:
print(num_total)
for i in range(11):
    print_fim(i)

In [ ]:


total_fim = np.zeros((5,5))
for i in range(314721):
    total_fim += calculator.fim_collection[i]
print(total_fim)

### Initialize

In [11]:
num_static = len(static_ind)
num_dynamic  = len(dynamic_ind)
num_total = num_static + num_dynamic*Nt

In [13]:
#with open('MarResults-A-strategy3/Mar20_A_110_8000_strategy3_mip', 'rb') as f:
with open('MarResult-strategy3-Dopt/Mar31_D_110_27700', 'rb') as f:
    x = pickle.load(f)
    
print(x)

init_cov_y = x.copy()

for i in range(11):
    for j in range(11):
        init_cov_y[i,j] = 1
    
# initialize all decisions
#init_cov_y = np.ones(x.shape) 

[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
for i in range(11,231):
    for j in range(11,231):
        init_cov_y[i,j] = 0

In [14]:
#with open('strategy3_all_covy', 'rb') as f:
with open('MarResult-strategy3-Dopt/Mar31_FIM_D_110_27700', 'rb') as f:
    fim_prior = pickle.load(f)
    
print(fim_prior)

[[ 5.55260011e+03 -2.61972445e+00 -4.14130933e+03  1.09879444e+04
  -5.00376769e+02]
 [-2.61972445e+00  5.22032483e+00  7.68731822e+00  5.03877570e+01
  -1.05219552e+01]
 [-4.14130933e+03  7.68731822e+00  3.32562801e+03 -8.52086707e+03
   1.66399892e+02]
 [ 1.09879444e+04  5.03877570e+01 -8.52086707e+03  2.52492460e+04
  -8.96253626e+02]
 [-5.00376769e+02 -1.05219552e+01  1.66399892e+02 -8.96253626e+02
   3.78315460e+02]]


In [137]:

mip_option = False
objective = ObjectiveLib.A
sparse_opt = True
fix_opt = False

manual_num = 20
budget_opt = 25000

num_dynamic_time = np.linspace(2,220,Nt)

static_dynamic = [[9,11], [10,12]]
#static_dynamic = None
time_interval_for_all = True

dynamic_time_dict = {}
for i, tim in enumerate(num_dynamic_time):
    dynamic_time_dict[i] = tim 

In [138]:
mod = calculator.continuous_optimization(mixed_integer=mip_option, 
                      obj=objective, 
                    fix=fix_opt, 
                    upper_diagonal_only=sparse_opt, 
                    num_dynamic_t_name = num_dynamic_time, 
                    manual_number = manual_num, 
                    budget=budget_opt,
                    init_cov_y= init_cov_y, 
                    initial_fim = fim_prior,
                    static_dynamic_pair=static_dynamic,
                    time_interval_all_dynamic = time_interval_for_all)

mod = calculator.solve(mod, mip_option=mip_option, objective = objective)

Ipopt 3.13.2: linear_solver=ma57


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        compu

  71 -3.4435062e+04 4.73e-11 1.03e+00  -5.7 8.90e+00    -  5.21e-02 3.65e-02f  1
  72 -3.4435692e+04 2.55e-11 1.11e+00  -5.7 7.89e+00    -  1.82e-01 4.04e-02f  1
  73 -3.4435849e+04 3.27e-11 1.31e+00  -5.7 7.34e+00    -  2.45e-01 1.07e-02f  1
  74 -3.4437387e+04 1.64e-11 1.07e+00  -5.7 5.72e+00    -  5.04e-02 1.22e-01f  1
  75 -3.4438196e+04 1.55e-11 1.03e+00  -5.7 5.22e+00    -  1.05e-01 7.08e-02f  1
  76 -3.4438307e+04 3.64e-11 1.15e+00  -5.7 4.76e+00    -  1.40e-01 1.05e-02f  1
  77 -3.4440911e+04 3.64e-11 9.41e-01  -5.7 7.42e+00    -  2.13e-01 1.82e-01f  1
  78 -3.4445499e+04 3.64e-11 5.75e-01  -5.7 6.21e+00    -  1.33e-01 3.86e-01f  1
  79 -3.4448173e+04 4.73e-11 3.70e-01  -5.7 3.97e+00    -  9.45e-02 3.57e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 -3.4450366e+04 4.37e-11 2.06e-01  -5.7 2.66e+00    -  2.03e-01 4.43e-01f  1
  81 -3.4451568e+04 5.46e-11 1.18e-01  -5.7 1.54e+00    -  3.80e-01 4.25e-01f  1
  82 -3.4452560e+04 3.64e-11

In [139]:
print(calculator.dynamic_install_cost)

[100, 100, 500, 500, 500]


In [140]:
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

ans_y, sol_y = calculator.extract_solutions(mod)
for i in range(5):
    print(all_names_strategy3[i+11])
    for t in range(len(sol_y[0])):
        if sol_y[i][t] > 0.95:
            print(dynamic_time_dict[t])
print('pyomo calculated cost:', pyo.value(mod.cost))

for i in [11,12,13,14,15]:
    print(pyo.value(mod.if_install_dynamic[i]))

[[ 5.54403778e+03 -3.14878630e+00 -4.14167461e+03  1.09711042e+04
  -4.90650301e+02]
 [-3.14878630e+00  5.17684301e+00  7.73505467e+00  4.93632935e+01
  -1.00031663e+01]
 [-4.14167461e+03  7.73505467e+00  3.32431860e+03 -8.51842177e+03
   1.65849584e+02]
 [ 1.09711042e+04  4.93632935e+01 -8.51842177e+03  2.52157288e+04
  -8.75799072e+02]
 [-4.90650301e+02 -1.00031663e+01  1.65849584e+02 -8.75799072e+02
   3.64836591e+02]]
trace: 34454.09860565377
det: 1961273837936.796
[3.31531757e+04 8.18876009e+02 4.33442110e+02 4.48920322e+01
 3.71274041e+00]
Pyomo OF: 34454.09860565377
Log_det: 28.30461529538087
Ads.gas_inlet.F :  1.0
Ads.gas_outlet.F :  1.0
Ads.gas_outlet.T :  1.0
Des.gas_inlet.F :  1.0
Des.gas_outlet.F :  1.0
Des.gas_outlet.T :  1.0
Ads.T_g.Value(19,10) :  1.0
Ads.T_g.Value(23,10) :  1.0
Ads.T_g.Value(28,10) :  1.0
Ads.gas_outlet.z("CO2").static :  1.0
Des.gas_outlet.z("CO2").static :  1.0
Ads.gas_outlet.z("CO2").dynamic
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [141]:
store = True

if store:
    file = open('./rotary_results/May12_'+str(budget_opt)+'_a', 'wb')

    pickle.dump(ans_y, file)

    file.close()
    
    file2 = open('./rotary_results/May12_fim_'+str(budget_opt)+'_a', 'wb')

    pickle.dump(fim_result, file2)

    file2.close()

In [116]:
# verify cost 
#real_cost = 0 
#for i in range(num_total):
#    real_cost += ans_y[i,i]*cost[i]
    
#dynamic_inst = []
#for j in range(num_static, num_dynamic+num_static):
#    dynamic_inst.append(pyo.value(mod.if_install_dynamic[j]))
#    for t in range(Nt):
#        if sol_y[j,t]>0.99:
#            dynamic_inst[j] = 1 
            
#real_cost += sum(dynamic_inst[i]*install_cost_dynamic[i] for i in range(num_dynamic))
#print(dynamic_inst)
#print('Real cost:', real_cost)


In [ ]:
count = 0 
for i in range(561):
    for j in range(i,561):
        #if pyo.value(mod.cov_y[i,j])<0.96:
        print(pyo.value(mod.cov_y[i,j]))
            #count += 1 
            
print(count)